In [1]:

import random
import time

# from google.colab import drive
from pathlib import Path
import pickle
import os


from functools import *
import pandas as pd
import gc
import collections
import copy

import nltk
nltk.download('stopwords')

import requests
import re
import json

[nltk_data] Downloading package stopwords to /Users/esben/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
parser = re.compile('\{\"tokens\": ')
def get_snippets(model_name, layer, neuron):
  """Get the max activating dataset examples for a given neuron in a model"""
  base_url = f"https://neuroscope.io/{model_name}/{layer}/{neuron}.html"

  response = requests.get(base_url)
  webpage = response.text
  
  parts = parser.split(webpage)
  snippets = []
  for i, part in enumerate(parts):    
    if i == 0 or i % 2 != 0:
      continue

    token_str = part.split(', "values": ')[0]

    tokens = json.loads(token_str)

    snippet = "".join(tokens)

    snippets.append(snippet)
    
  if len(snippets) != 20:
    raise Exception
  return snippets

In [3]:
snippets = get_snippets("solu-6l-pile", 3, 13)

In [4]:
act_parser = re.compile('<h4>Max Act: <b>')
def get_max_activations(model_name, layer, neuron, n=1):
  """Get the max activating dataset examples for a given neuron in a model"""
  base_url = f"https://neuroscope.io/{model_name}/{layer}/{neuron}.html"

  response = requests.get(base_url)
  webpage = response.text
  
  parts = act_parser.split(webpage)
  activations = []
  for i, part in enumerate(parts):    
    if i == 0:
      continue

    activation = float(part.split('</b>')[0])

    activations.append(activation)
    if len(activations) >= n:
      break
    
  if len(activations) != min(20, n):
    raise Exception
  return activations if n > 1 else activations[0]

## Scrape max activations

In [5]:
def get_max_acts(model_name, layer_and_neurons):
  layer, neurons = layer_and_neurons
  activations = []
  for i, neuron in enumerate(neurons):
    if i % 50 == 0:
      print(f"\nLayer {layer}: {i} of {len(neurons)} complete")
    try:
      activation = get_max_activations(model_name, layer, neuron, n=1)
      activations.append(activation)
    except:
      print(f"Neuron {neuron} in layer {layer} failed")  
      # Use the previous activation as a hack to get around failures
      activations.append(activations[-1])  
  return activations

In [6]:
import multiprocessing as mp

layers = 48
neurons = 6400
model_name = "gpt2-xl"

info = [(layer, [neuron for neuron in range(neurons)]) for layer in range(layers)]

In [7]:
%%time

with mp.Pool(layers) as p:
    activation_matrix = p.map(partial(get_max_acts, model_name), info)

Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Users/esben/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/esben/anaconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/esben/anaconda3/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/esben/anaconda3/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'get_max_acts' on <module '__main__' (built-in)>
Process SpawnPoolWorker-6:
Process SpawnPoolWorker-7:
Traceback (most recent call last):
  File "/Users/esben/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/esben/anaconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._targ

KeyboardInterrupt: 

In [ ]:
activation_matrix_np = np.array(activation_matrix)
activation_matrix_np.shape

In [ ]:
import json

if True:
  with open(os.path.join(base_path, f"data/activation_matrix-{model_name}.json"), "w") as ofh:
    json.dump(activation_matrix, ofh, indent=2, ensure_ascii=False)

In [ ]:
import math


def batch(arr, n=None, batch_size=None):
    if n is None and batch_size is None:
        raise ValueError("Either n or batch_size must be provided")
    if n is not None and batch_size is not None:
        raise ValueError("Either n or batch_size must be provided, not both")

    if n is not None:
        batch_size = math.floor(len(arr) / n)
    elif batch_size is not None:
        n = math.ceil(len(arr) / batch_size)

    extras = len(arr) - (batch_size * n)
    groups = []
    group = []
    added_extra = False
    for element in arr:
        group.append(element)
        if len(group) >= batch_size:
            if extras and not added_extra:
                extras -= 1
                added_extra = True
                continue
            groups.append(group)
            group = []
            added_extra = False

    if group:
        groups.append(group)

    return groups

In [ ]:
from datasets import load_dataset

data_files = {"validation": "en/c4-validation.*.json.gz"}
c4_validation = load_dataset("allenai/c4", data_files=data_files, split="validation")

Method
- Scrape max activation of neuron on any token from lexoscope
- Create layer x neuron matrix of max activations * scale to get activation thresholds
- Threshold the activation matrix for a given example with the threshold matrix
- Create a dict/matrix where, for each neuron in the model, there is an array containing a tokenized text example and the actvation of the neuron on each token. If this takes too much space, could just store the activations and the index of the text example, and separately store the tokenized text examples by index
- (Store the non-zero entries in a dict, where each key is the index of an example, and each value is a tuple with the example text and an array of dicts mapping (layer, neuron) indices to the activation for each token in the text) - old idea

In [ ]:
%%time
from tqdm.autonotebook import tqdm
import gzip    

layer_indices = [i for i in range(6)]

all_activation_groups = []
prompts = []
prompt_count = 0

process_data = batched[:50000]

if False:
  with gzip.open(os.path.join(base_path, "data/activation_data.jsonl.gz"), "wb") as ofh1, open(os.path.join(base_path, "data/prompts.jsonl"), "w") as ofh2:
    for i, data_batch in tqdm(enumerate(process_data), total=len(process_data), desc="Progress"):
      tokens = model.to_tokens(data_batch)
      str_tokens = model.to_str_tokens(data_batch)
      logits, cache = model.run_with_cache(tokens)
      layer_activations = torch.stack([cache[f"blocks.{layer_index}.mlp.hook_mid"] for layer_index in layer_indices]).permute(1, 2, 0, 3).cpu().numpy()
      # print(layer_activations)
      # print(threshold_matrix)
      thresholded = layer_activations > threshold_matrix
      strong_activations = np.where(layer_activations > threshold_matrix, layer_activations, 0)
      strong_activations /= activation_matrix
      strong_activations = strong_activations.transpose(0, 2, 3, 1)

      for j, batch_activations in enumerate(strong_activations):
        layer_groups = []
        for k, layer_activations in enumerate(batch_activations):
          neuron_groups = {}
          for l, neuron_activations in enumerate(layer_activations):  
            non_zero_indices = np.nonzero(neuron_activations)[0]
            if len(non_zero_indices) == 0:
              continue
            token_activations = {int(index): float(neuron_activations[index]) for index in non_zero_indices}
            group = {"prompt_index": prompt_count, "token_activations": token_activations}
            # print(len(group))
            neuron_groups[l] = group
          # print(len(neuron_groups))
          layer_groups.append(neuron_groups)
        # print(len(layer_groups))
        ofh1.write(f"{json.dumps(layer_groups)}\n".encode())
        ofh2.write(f"{json.dumps(str_tokens[j])}\n")
        prompt_count += 1

      # print(len(all_activation_groups))
      # print(tokens.shape)
      # print(layer_activations.shape)
      # print(threshold_matrix.shape)
      # print(thresholded.shape)
      # print(strong_activations.shape)
      # print(np.count_nonzero(strong_activations))
      # break

  with open(os.path.join(base_path, "data/prompts.json"), "w") as ofh:
    json.dump(prompts, ofh, ensure_ascii=False)